# Fremont Bridge Bicycle Counter

> The Fremont Bridge Bicycle Counter began operation in October 2012 and records the number of bikes that cross the bridge using the pedestrian/bicycle pathways. Inductive loops on the east and west pathways count the passing of bicycles regardless of travel direction. The data consists of a date/time field: Date, east pathway count field: Fremont Bridge NB, and west pathway count field: Fremont Bridge SB. The count fields represent the total bicycles detected during the specified one hour period. **Direction of travel is not specified, but in general most traffic in the Fremont Bridge NB field is travelling northbound and most traffic in the Fremont Bridge SB field is travelling southbound.**

<img src="fremont_bridge.png" alt="fremont bridge" width="200"/>

https://data.seattle.gov/Transportation/Fremont-Bridge-Bicycle-Counter/65db-xm6k

In [1]:
import datetime
import json
import pathlib
from urllib.parse import urlencode
import time
import random

import altair as alt
import pandas as pd
import requests

pd.set_option('display.max_columns', 50)

In [2]:
PAGES_DIR = pathlib.Path('weather_pages')

BASE_URL = 'https://data.seattle.gov/resource/65db-xm6k.json'
START_DATE = datetime.datetime(2022, 5, 1)
END_DATE = datetime.datetime(2022, 6, 1)

## weather data

In [3]:
WEATHER_URL = 'https://api.weather.com/v1/location/KSEA:9:US/observations/historical.json'

PARAMS = {
    'apiKey': 'e1f10a1e78da46f5b10a1e78da96f525',
    'units': 'e',
    'startDate': None,
    'endDate': None,
}

In [4]:
dates = [datetime.datetime(2022, 5, i).strftime('%Y%m%d') for i in range(1, 32)]

for date in dates:
    fout = PAGES_DIR / f'{date}.json'

    # will only download once
    if not fout.exists():
        print(fout.name)

        params = PARAMS.copy()
        params['startDate'] = date
        params['endDate'] = date
        url = f'{WEATHER_URL}?{urlencode(params)}'

        r = requests.get(url)
        with open(fout, 'wt') as f:
            json.dump(r.json(), f, indent=2)

        time.sleep(random.random() * 3 + 3)

In [5]:
days = []
for fname in PAGES_DIR.glob('*.json'):
    with open(fname) as f:
        weather_data = json.load(f)
        observations = weather_data.get('observations')  # assumed to be ordered
        days.append(pd.DataFrame.from_dict(observations))

weather = pd.concat(days)
weather['valid_time_gmt'] = weather['valid_time_gmt'].apply(lambda ts: datetime.datetime.fromtimestamp(ts))
weather['expire_time_gmt'] = weather['expire_time_gmt'].apply(lambda ts: datetime.datetime.fromtimestamp(ts))
weather['date_truncated'] = pd.to_datetime(weather['valid_time_gmt'].dt.strftime('%Y-%m-%dT%H:00:00'))

weather.head(2)

,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,wx_phrase,pressure_tend,pressure_desc,dewPt,heat_index,rh,pressure,vis,wc,wdir,wdir_cardinal,gust,wspd,max_temp,min_temp,precip_total,precip_hrly,snow_hrly,uv_desc,feels_like,uv_index,qualifier,qualifier_svrty,blunt_phrase,terse_phrase,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction,date_truncated
0,KSEA,observation,2022-05-07 02:10:00,KSEA,Seattle,2022-05-07 00:10:00,N,47,11,1201,Light Rain,NaN,None,45,47,93,29.19,5.0,47.0,320.0,NW,NaN,3.0,NaN,NaN,NaN,0.03,None,Low,47.0,0,None,None,None,None,OVC,None,None,None,None,None,None,None,None,None,2022-05-07
1,KSEA,observation,2022-05-07 02:22:00,KSEA,Seattle,2022-05-07 00:22:00,N,47,11,1201,Light Rain,NaN,None,45,47,93,29.19,7.0,47.0,300.0,WNW,NaN,3.0,NaN,NaN,NaN,0.03,None,Low,47.0,0,None,None,None,None,OVC,None,None,None,None,None,None,None,None,None,2022-05-07


Truncate date by day/hour

In [6]:
weather_by_hour = (weather.groupby('date_truncated')['temp']
                          .mean().round(1).reset_index()
                          .rename(columns={'date_truncated': 'date'}))

# len(weather_by_hour)
weather_by_hour.head(2)

,date,temp
0,2022-05-01 00:00:00,47.0
1,2022-05-01 01:00:00,46.0


## bike count data

filtering:

* https://dev.socrata.com/foundry/data.seattle.gov/65db-xm6k
* https://dev.socrata.com/docs/functions/between.html

In [7]:
def bike_counts_by_date_range(start: datetime.datetime, end: datetime.datetime) -> list[dict]:
    param = f"$where=date between '{start.isoformat()}' and '{end.isoformat()}'"
    url = f'{BASE_URL}?{param}'
    r = requests.get(url)

    return r.json()

In [8]:
def parse_df(bike_counts: list[dict]) -> pd.DataFrame:
    """Process bike count data"""
    df = pd.DataFrame.from_dict(bike_counts)
    df['date'] = pd.to_datetime(df['date'])
    df['dow'] = df['date'].dt.day_name()

    for col in ['fremont_bridge', 'fremont_bridge_nb', 'fremont_bridge_sb']:
        df[col] = df[col].astype(int)

    return df

### load bike data

In [9]:
fname = pathlib.Path().cwd() / 'bike_counts.parquet'
if fname.exists():
    bike_counts = pd.read_parquet(fname)
else:
    bike_counts_raw = bike_counts_by_date_range(START_DATE, END_DATE)
    bike_counts = parse_df(bike_counts_raw)
    df.to_parquet('bike_counts.parquet')

# should be 1 row per hour everyday for 31 days
assert ((bike_counts['fremont_bridge'] != (bike_counts['fremont_bridge_nb'] + bike_counts['fremont_bridge_sb'])).sum() == 0)

# make sure all totals equal NB + SB
assert len(bike_counts) == 31 * 24

bike_counts.head()

,date,fremont_bridge,fremont_bridge_sb,fremont_bridge_nb,dow
0,2022-05-01 00:00:00,14,6,8,Sunday
1,2022-05-01 01:00:00,9,3,6,Sunday
2,2022-05-01 02:00:00,6,2,4,Sunday
3,2022-05-01 03:00:00,2,0,2,Sunday
4,2022-05-01 04:00:00,4,1,3,Sunday


In [10]:
print(f"Date range: {bike_counts['date'].min()} to {bike_counts['date'].max()}")

Date range: 2022-05-01 00:00:00 to 2022-05-31 23:00:00


## Charts

In [11]:
alt.Chart(bike_counts).mark_line().encode(x='date', y='fremont_bridge', tooltip=['dow', 'fremont_bridge']).interactive()

alt.Chart(...)

In [12]:
df = pd.merge(bike_counts, weather_by_hour, on='date')

df.head()

,date,fremont_bridge,fremont_bridge_sb,fremont_bridge_nb,dow,temp
0,2022-05-01 00:00:00,14,6,8,Sunday,47.0
1,2022-05-01 01:00:00,9,3,6,Sunday,46.0
2,2022-05-01 02:00:00,6,2,4,Sunday,47.0
3,2022-05-01 03:00:00,2,0,2,Sunday,46.0
4,2022-05-01 04:00:00,4,1,3,Sunday,45.0


In [13]:
chart = (alt.Chart(df).mark_circle()
            .encode(x=alt.X('temp', scale=alt.Scale(domain=[35, 75])), y='fremont_bridge', color='dow', size='temp', tooltip=['dow', 'temp'])
            .interactive())

chart.properties(title='Fremont bridge total bike crossings - May 2022')

alt.Chart(...)